# Applied Data Science Capstone

## Introduction

In my scenario, I am interested in finding an ideal neighborhood for a late-night French Fry Boutique. Even in Atlanta, there are few late night food options and there is potential for demand. This hypothetical french fry boutique would feature items ranging from about $5-15 and we will focus on snack and comfort food rather than entrees, for the sake of defining our business.

To guide enquiry, we will lead with a few hypotheses and verify them as best as we can with available data.
### Positive Influencers
* Proximity to younger areas, high schools, and colleges
* Affluent area
* Local bars and late-night attractions
* Lots of pedestrians
    * Area should be known as generally safe
    * Pedestrians new to the store may be more willing to stop in and grab a snack  
    
### Negative Influencers  

* Fast Food
    * May lose potential new customers if they compare our fries 'with the works' to a chain's $0.99 box of fries

## Data

We will be using several datasets to evaluate our hypotheses above and answer our ultimate question.
* [GeoJson](https://github.com/blackmad/neighborhoods) to map the neighborhood borders in Folium
* Data concerning various statistics about the neighborhoods
    * I have already scraped the data from [here](http://www.city-data.com/nbmaps/neigh-Atlanta-Georgia.html) and saved it to a csv file 'atl_neighborhoods.csv"
* Data gathered from Foursquare
    * Discover information about potential competitors to confirm, deny, or modify our hypotheses
    * Determine companion shops via clustering to generate viable neighborhoods
* [Crime data](http://www.atlantapd.org/i-want-to/crime-data-downloads) to evaluate safety